In [2]:
from google.colab import files 
uploaded = files.upload() 

Saving claimskg_result.csv to claimskg_result.csv


In [3]:
import io 
import pandas as pd
data = pd.read_csv(io.BytesIO(uploaded['claimskg_result.csv'])) 
print(type(data))
print(data.head())

<class 'pandas.core.frame.DataFrame'>
                                                  id  ... language
0  http://data.gesis.org/claimskg/claim_review/b0...  ...  English
1  http://data.gesis.org/claimskg/claim_review/03...  ...  English
2  http://data.gesis.org/claimskg/claim_review/e1...  ...  English
3  http://data.gesis.org/claimskg/claim_review/3c...  ...  English
4  http://data.gesis.org/claimskg/claim_review/b4...  ...  English

[5 rows x 14 columns]


In [0]:
data.drop(["id","text","date","truthRating","author","headline","keywords","sourceURL","link","language"],axis=1,inplace=True)

In [5]:
data.columns

Index(['ratingName', 'named_entities_claim', 'named_entities_article',
       'source'],
      dtype='object')

In [6]:
data.shape

(10000, 4)

In [7]:
clean_data=data.dropna()
clean_data.head()

,ratingName,named_entities_claim,named_entities_article,source
0,False,"Netflix,Netflix and Chill,Republican,Ted Cruz,...","Netflix and Chill,Ted Cruz",snopes
2,False,"ABCNews.com,California Department of Public He...","California Department of Public Health,measles",snopes
3,False,"2008. Democratic presidential nominee,Alfred...","Michelle Obama,Waldorf-Astoria hotel,room service",snopes
4,True,"Associated Press,Bonkers,Facebook,Herendeen,Ne...",Silly Bandz,snopes
5,False,"Bay of Fundy,Brine Shrimp,Brine shrimp,Kittery...","brine shrimp,lobster,vagina",snopes


In [8]:
clean_data.shape

(6656, 4)

In [9]:
data_x=clean_data.iloc[:,1:]
data_x.head()

,named_entities_claim,named_entities_article,source
0,"Netflix,Netflix and Chill,Republican,Ted Cruz,...","Netflix and Chill,Ted Cruz",snopes
2,"ABCNews.com,California Department of Public He...","California Department of Public Health,measles",snopes
3,"2008. Democratic presidential nominee,Alfred...","Michelle Obama,Waldorf-Astoria hotel,room service",snopes
4,"Associated Press,Bonkers,Facebook,Herendeen,Ne...",Silly Bandz,snopes
5,"Bay of Fundy,Brine Shrimp,Brine shrimp,Kittery...","brine shrimp,lobster,vagina",snopes


In [10]:
data_y=clean_data.iloc[:,:1]
data_y.head()

,ratingName
0,False
2,False
3,False
4,True
5,False


In [11]:
print(data_x.shape)
print(data_y.shape)

(6656, 3)
(6656, 1)


In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier

In [0]:
def split_data(x,y):
    
    """
    Reminder : Clean the set before sending here.
    Input  : x=Attributes of the dataset, y=Target Attribute
    Output : Split data of train test
    
    We are removing dev , as dev can be better used for the models that includes DNN, CNN and RNN
    
    """
    X=x
    Y=y
    
    train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=0.2,random_state=12)
    
    #train_x : Training data with attributes
    #train_y : Training data with ratings
    #test_x  : testing data with attributes
    #test_y  : testing data with rating 
    
    print("Final size of train/test :",train_x.size,"/",test_x.size)
    return train_x,train_y,test_x,test_y

In [14]:
import numpy as np
a=np.array([])
a.size

0

In [0]:
def passive_aggressive(train_x,train_y,test_x,test_y,stop=False):
    
    if stop==True:
        print("Early Stopping is being used")
    
    pac= PassiveAggressiveClassifier(early_stopping=stop)
    pac.fit(train_x, train_y)
    predicted_pac_train=pac.predict(train_x)
    predicted_pac_test= pac.predict(test_x)
    
    score_train= metrics.accuracy_score(train_y, predicted_pac_train)
    score_test=metrics.accuracy_score(test_y,predicted_pac_test)
    
    print("train accuracy:",score_train)
    cm_pac_train= metrics.confusion_matrix(train_y, predicted_pac_train)
    print("train confusion matrix")
    print(cm_pac_train)
    print("test accuracy:",score_test)
    cm_pac_test= metrics.confusion_matrix(test_y, predicted_pac_test)
    print("test confusion matrix")
    print(cm_pac_test)

In [0]:
def mult_tfidf_pac(x1,x2,y,x3=a,stop=False):
    from sklearn.feature_extraction.text import TfidfVectorizer

    if x3.size==0:
        train_x1,train_y1,test_x1,test_y1=split_data(x1,y)
        train_x2,train_y2,test_x2,test_y2=split_data(x2,y)



        tfidf_vectorizer = TfidfVectorizer(stop_words='english',max_df=0.7,min_df=0.02)   # a TFIDF vectorizer
        tfidf_train_x1 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x1).todense())  #fitting the training data
        tfidf_test_x1 = pd.DataFrame(tfidf_vectorizer.transform(test_x1).todense())

        tfidf_train_x2 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x2).todense())  #fitting the training data
        tfidf_test_x2 = pd.DataFrame(tfidf_vectorizer.transform(test_x2).todense()) 

        train=pd.concat([tfidf_train_x1,tfidf_train_x2],axis=1)
        test=pd.concat([tfidf_test_x1,tfidf_test_x2],axis=1)
        print(train.shape)
        print(test.shape)
        print(train_y2.shape)
        print(test_y2.shape)

        passive_aggressive(tfidf_train_x1,train_y1,tfidf_test_x1,test_y1)
    
    else:
        train_x1,train_y1,test_x1,test_y1=split_data(x1,y)
        train_x2,train_y2,test_x2,test_y2=split_data(x2,y)
        train_x3,train_y3,test_x3,test_y3=split_data(x3,y)
        
        tfidf_vectorizer = TfidfVectorizer(stop_words='english',max_df=0.7,min_df=0.02)   # a TFIDF vectorizer
        tfidf_train_x1 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x1).todense())  #fitting the training data
        tfidf_test_x1 = pd.DataFrame(tfidf_vectorizer.transform(test_x1).todense())

        tfidf_train_x2 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x2).todense())  #fitting the training data
        tfidf_test_x2 = pd.DataFrame(tfidf_vectorizer.transform(test_x2).todense()) 
        
        tfidf_train_x3 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x3).todense())  #fitting the training data
        tfidf_test_x3 = pd.DataFrame(tfidf_vectorizer.transform(test_x3).todense())
        
        train=pd.concat([tfidf_train_x1,tfidf_train_x2,tfidf_train_x3],axis=1)
        test=pd.concat([tfidf_test_x1,tfidf_test_x2,tfidf_test_x3],axis=1)
        
        passive_aggressive(train,train_y2,test,test_y2)

In [17]:
mult_tfidf_pac(x1=data_x.named_entities_article,y=data_y.ratingName,stop=True) #single attr named_entities_article with early stopping

Final size of train/test : 5324 / 1332
(5324, 7)
(1332, 7)
train accuracy: 0.7667167543200601
train confusion matrix
[[4082    0]
 [1242    0]]
test accuracy: 0.7672672672672672
test confusion matrix
[[1022    0]
 [ 310    0]]


In [18]:
mult_tfidf_pac(x1=data_x.named_entities_claim,y=data_y.ratingName,stop=True) #single attr n_e_claim with early stop

Final size of train/test : 5324 / 1332
(5324, 153)
(1332, 153)
train accuracy: 0.7638993238166792
train confusion matrix
[[3949  133]
 [1124  118]]
test accuracy: 0.7612612612612613
test confusion matrix
[[986  36]
 [282  28]]


In [19]:
mult_tfidf_pac(x1=data_x.source,y=data_y.ratingName,stop=True) #single attr source with early stop

Final size of train/test : 5324 / 1332
(5324, 4)
(1332, 4)
train accuracy: 0.4154770848985725
train confusion matrix
[[1487 2595]
 [ 517  725]]
test accuracy: 0.4084084084084084
test confusion matrix
[[374 648]
 [140 170]]


In [20]:
mult_tfidf_pac(x1=data_x.named_entities_article,y=data_y.ratingName,stop=False) #single attr n_e_ar without early stop

Final size of train/test : 5324 / 1332
(5324, 7)
(1332, 7)
train accuracy: 0.7036063110443276
train confusion matrix
[[3661  421]
 [1157   85]]
test accuracy: 0.704954954954955
test confusion matrix
[[914 108]
 [285  25]]


In [21]:
mult_tfidf_pac(x1=data_x.named_entities_claim,y=data_y.ratingName,stop=False) #single attr n_e_c without early stop

Final size of train/test : 5324 / 1332
(5324, 153)
(1332, 153)
train accuracy: 0.768595041322314
train confusion matrix
[[4019   63]
 [1169   73]]
test accuracy: 0.7612612612612613
test confusion matrix
[[1001   21]
 [ 297   13]]


In [22]:
mult_tfidf_pac(x1=data_x.source,y=data_y.ratingName,stop=False) #single attr source without early stop

Final size of train/test : 5324 / 1332
(5324, 4)
(1332, 4)
train accuracy: 0.7667167543200601
train confusion matrix
[[4082    0]
 [1242    0]]
test accuracy: 0.7672672672672672
test confusion matrix
[[1022    0]
 [ 310    0]]


In [24]:
mult_tfidf_pac(x1=data_x.named_entities_article,x2=data_x.named_entities_claim,y=data_y.ratingName,x3=a,stop=True) #n_e_a + n_e_c with early stop

Final size of train/test : 5324 / 1332
Final size of train/test : 5324 / 1332
(5324, 160)
(1332, 160)
(5324,)
(1332,)
train accuracy: 0.7667167543200601
train confusion matrix
[[4082    0]
 [1242    0]]
test accuracy: 0.7672672672672672
test confusion matrix
[[1022    0]
 [ 310    0]]


In [25]:
mult_tfidf_pac(x1=data_x.named_entities_article,x2=data_x.source,y=data_y.ratingName,x3=a,stop=True) #n_e_a + source with early stop

Final size of train/test : 5324 / 1332
Final size of train/test : 5324 / 1332
(5324, 11)
(1332, 11)
(5324,)
(1332,)
train accuracy: 0.7667167543200601
train confusion matrix
[[4082    0]
 [1242    0]]
test accuracy: 0.7672672672672672
test confusion matrix
[[1022    0]
 [ 310    0]]


In [26]:
mult_tfidf_pac(x1=data_x.named_entities_claim,x2=data_x.source,y=data_y.ratingName,x3=a,stop=True) #n_e_c +s ource with early stop

Final size of train/test : 5324 / 1332
Final size of train/test : 5324 / 1332
(5324, 157)
(1332, 157)
(5324,)
(1332,)
train accuracy: 0.7436138241923366
train confusion matrix
[[3691  391]
 [ 974  268]]
test accuracy: 0.7282282282282282
test confusion matrix
[[922 100]
 [262  48]]


In [28]:
mult_tfidf_pac(x1=data_x.named_entities_article,x2=data_x.source,y=data_y.ratingName,x3=data_x.named_entities_claim,stop=True) #all 3 with early stop

Final size of train/test : 5324 / 1332
Final size of train/test : 5324 / 1332
Final size of train/test : 5324 / 1332
train accuracy: 0.6604057099924868
train confusion matrix
[[2969 1113]
 [ 695  547]]
test accuracy: 0.6486486486486487
test confusion matrix
[[735 287]
 [181 129]]


In [29]:
mult_tfidf_pac(x1=data_x.named_entities_article,x2=data_x.named_entities_claim,y=data_y.ratingName,x3=a,stop=False) #n_e_a + n_e_c without early stop

Final size of train/test : 5324 / 1332
Final size of train/test : 5324 / 1332
(5324, 160)
(1332, 160)
(5324,)
(1332,)
train accuracy: 0.7601427498121713
train confusion matrix
[[4015   67]
 [1210   32]]
test accuracy: 0.762012012012012
test confusion matrix
[[1008   14]
 [ 303    7]]


In [30]:
mult_tfidf_pac(x1=data_x.named_entities_article,x2=data_x.source,y=data_y.ratingName,x3=a,stop=False) #n_e_a + source without early stop

Final size of train/test : 5324 / 1332
Final size of train/test : 5324 / 1332
(5324, 11)
(1332, 11)
(5324,)
(1332,)
train accuracy: 0.7667167543200601
train confusion matrix
[[4082    0]
 [1242    0]]
test accuracy: 0.7672672672672672
test confusion matrix
[[1022    0]
 [ 310    0]]


In [31]:
mult_tfidf_pac(x1=data_x.named_entities_claim,x2=data_x.source,y=data_y.ratingName,x3=a,stop=False) #n_e_c +s ource without early stop

Final size of train/test : 5324 / 1332
Final size of train/test : 5324 / 1332
(5324, 157)
(1332, 157)
(5324,)
(1332,)
train accuracy: 0.7351615326821939
train confusion matrix
[[3543  539]
 [ 871  371]]
test accuracy: 0.7027027027027027
test confusion matrix
[[866 156]
 [240  70]]


In [32]:
mult_tfidf_pac(x1=data_x.named_entities_article,x2=data_x.source,y=data_y.ratingName,x3=data_x.named_entities_claim,stop=True) #all 3 with early stop

Final size of train/test : 5324 / 1332
Final size of train/test : 5324 / 1332
Final size of train/test : 5324 / 1332
train accuracy: 0.7648384673178061
train confusion matrix
[[4019   63]
 [1189   53]]
test accuracy: 0.7605105105105106
test confusion matrix
[[1008   14]
 [ 305    5]]
